Transformer with only encoder and decoder block that can translate English text to Arabic, given one file that contains English and its translated Arabic text

# Importing Required Library

In [ ]:
import pandas as pd
from tensorflow.keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reading Data and converting it to DataFrame

In [ ]:
English = []
Arabic = []
with open('/content/ara_eng.txt', 'r',encoding='utf-8') as file:
     lines = file.read().split("\n")[:-1]
     for line in lines:
       English.append(line.split("\t")[0])
       Arabic.append(line.split("\t")[1])

data = {"English": English, "Arabic": Arabic}
df = pd.DataFrame(data)
df = df.iloc[0:10000]
df["Arabic"] = "<start>" + df["Arabic"] + "<end>"

# Maximum Sequence Length

In [ ]:
Sequence_Length = 0
for sequence in df['Arabic']:
    sequence = sequence.split()
    if  len(sequence) > Sequence_Length:
        Sequence_Length = len(sequence)


# Vocab size

In [ ]:
text_data = " ".join(df['English'].values.tolist())
English_Vocab_Size = text_data.split()
English_Vocab_Size = len(set(English_Vocab_Size))

text_data = " ".join(df['Arabic'].values.tolist())
Arabic_Vocab_Size = text_data.split()
Arabic_Vocab_Size = len(set(Arabic_Vocab_Size))
Vocab_Size = English_Vocab_Size + Arabic_Vocab_Size

# This code takes the values in the 'English' column of the DataFrame df and joins them into a single string, text_data. 
# Then, the string is split into individual words using the .split() method. 
# By converting the resulting list into a set and calculating its length, you obtain the number of unique words in the English language present in the DataFrame. 
# This value is stored in the English_Vocab_Size variable.

#  Text Vectorization 

In [ ]:
# These layers are used to convert text into sequences of integers, which can be fed into the neural network models.

English_Vectorization = TextVectorization(max_tokens=Vocab_Size, output_mode="int", output_sequence_length=Sequence_Length,)

Arabic_Vectorization = TextVectorization(max_tokens=Vocab_Size,output_mode="int",output_sequence_length=Sequence_Length + 1,)
# The additional +1 is likely because the target sequences were mentioned to be shifted by one timestep.

# Splitting Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['English'],df['Arabic'] , test_size=0.1, random_state=0)

# Dataset Preparation and Formatting

In [ ]:
English_Text = [text for text in X_train]
Arabic_Text = [text for text in y_train]
# Each element in the list represents a text sample.

English_Vectorization.adapt(English_Text)
Arabic_Vectorization.adapt(Arabic_Text)
# This step analyzes the text data and builds the vocabulary based on the maximum number of tokens specified during the initialization of the vectorization layers. 
#It also sets up the internal mapping between words and integer indices.

In [ ]:
def format_dataset(English, Arabic):
    English = English_Vectorization(English)  
    Arabic = Arabic_Vectorization(Arabic)  
    return ({"encoder_inputs": English, "decoder_inputs": Arabic[:, :-1]}, Arabic[:, 1:])
# The function returns a tuple containing the preprocessed input and target data for the model.

dataset = tf.data.Dataset.from_tensor_slices((English_Text, Arabic_Text))  #This line creates a TensorFlow dataset, It combines the English and Arabic text data into a tuple of tensors, where each element of the tuple represents a sample pair.
batch_size = 128  # used to group the individual samples into batches of size 128
dataset = dataset.batch(batch_size)
dataset = dataset.map(format_dataset)  # This function converts the text sequences into integer sequences using the previously adapted TextVectorization layers.

train = dataset.take(len(X_train))  # The take() method is used to create a new dataset containing the first len(X_train) samples
test = dataset.skip(len(X_train))   # The skip() method is then used to create a new dataset containing the remaining samples, representing the test set.

# Building a Transformer-based Neural Machine Translation Model with TensorFlow

In [ ]:
# The code snippet you provided defines three custom layers for the Transformer model: TransformerEncoder, PositionalEmbedding, and TransformerDecoder. 
# These layers are building blocks of the Transformer architecture used in sequence-to-sequence tasks like machine translation.
# Each layer defines its own call() method, which defines the forward pass logic, and some layers define a compute_mask() method to handle masking. 
# These layers can be used to construct the Transformer model for sequence-to-sequence tasks.

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
   # applies multi-head self-attention mechanism, projects the attention output, and adds residual connections and layer normalization. It supports masking to handle padded sequences.

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
   # It consists of two embedding layers: one for token embeddings and another for positional embeddings. 
   # It combines the embeddings by adding them element-wise. The positional embeddings represent the position of each token in the sequence.

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    # applies multi-head self-attention mechanism and encoder-decoder attention mechanism, projects the attention output, and adds residual connections and layer normalization. 
    # It also supports masking, including a causal attention mask for the self-attention mechanism to attend only to previous positions.

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult) # This line tiles (replicates) the mask tensor batch_size times along the batch axis using the multiplication tensor 'mult'  and represents the causal attention mask.
    # this function is used within the call method of the TransformerDecoder class to apply the causal attention mask during the self-attention mechanism.
    # generates a causal attention mask for the self-attention mechanism in the decoder. 
    # This mask ensures that each position in the decoder attends only to the previous positions, preventing information flow from future positions during training.

In [ ]:
#  It defines separate models for the encoder and decoder and then combines them into a single Transformer model.

embed_dim = 256  # The dimensionality of the token embeddings and positional embeddings.
latent_dim = 1024  # The dimensionality of the intermediate dense layer in the TransformerDecoder.
num_heads = 4  # The number of attention heads in the multi-head attention layers.

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(Sequence_Length, Vocab_Size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(Sequence_Length, Vocab_Size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.1)(x)  # Apply dropout regularization to the decoder outputs
decoder_outputs = layers.Dense(Vocab_Size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 30  

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train, epochs=epochs, validation_data=test)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   4888832     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   1578496     ['positional_embedding[